In [ ]:
!pip install keras==2.4.3
!pip install tensorflow

import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from keras.models import Model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


# Read the CSV file into a pandas DataFrame
imdb_data = pd.read_csv('IMDB Dataset.csv')

# Split the IMDb data into train and test sets
train_data, test_data = train_test_split(imdb_data, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
X_train = tokenizer.texts_to_sequences(train_data['review'])
X_test = tokenizer.texts_to_sequences(test_data['review'])

# Pad the sequences to ensure they are all the same length
max_length = 150
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

# Convert the labels to numerical values
y_train = train_data['sentiment'].map({'positive': 1, 'negative': 0}).values
y_test = test_data['sentiment'].map({'positive': 1, 'negative': 0}).values

# Define the model architecture
inputs = Input(shape=(max_length,))
embedding_layer = Embedding(5000, 150, input_length=max_length)(inputs)
conv_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(embedding_layer)
pooling_layer = MaxPooling1D(pool_size=2)(conv_layer)
bi_lstm_layer = Bidirectional(LSTM(64))(pooling_layer)
dropout_layer = Dropout(0.5)(bi_lstm_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)
model = Model(inputs=inputs, outputs=output_layer)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).flatten()
acc = accuracy_score(y_test, y_pred)
print('Accuracy:', acc)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.4.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


Epoch 1/3
625/625 [==============================] - 134s 206ms/step - loss: 0.3662 - accuracy: 0.8275 - val_loss: 0.2881 - val_accuracy: 0.8810
Epoch 2/3
625/625 [==============================] - 138s 220ms/step - loss: 0.2370 - accuracy: 0.9075 - val_loss: 0.2524 - val_accuracy: 0.8923
Epoch 3/3
313/313 [==============================] - 11s 31ms/step
Accuracy: 0.8918
